In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math,copy

In [ ]:
df = pd.read_csv('student_dataset.csv')
x_train = df[['Hours of Study','Previous Exam %','IQ','Attendance %','Average Quiz Score']].to_numpy()
y_train = df['Exam Result'].to_numpy().reshape(-1,1)
print(f"X shape: {x_train.shape}")
print(f"Y shape: {y_train.shape}")

X shape: (102, 5)
Y shape: (102, 1)


In [13]:
def sigmoid(z):
    g = 1/(1+np.exp(-z))
    return g

In [11]:
def z_score_normalization(x):
    mu = np.mean(x,axis=0)
    sigma = np.std(x,axis=0)
    x_norm = (x-mu)/sigma
    return x_norm

In [29]:
x_train = z_score_normalization(x_train)

In [30]:
def compute_logistic_cost_reg(x,y,w,b,lambdaa):
    m,n = x.shape
    cost = 0.0
    reg_cost = 0.0
    for i in range(m):
        z_i = np.dot(x[i],w)+b
        f_wb_i = sigmoid(z_i)
        cost += ((f_wb_i - y[i])**2)
    cost = cost/(2*m)
    for j in range(n):
        reg_cost += (w[j]**2)
    reg_cost = (lambdaa/(2*m))*reg_cost
    reg_cost = cost + reg_cost
    return reg_cost

In [18]:
def compute_logistic_gradient_reg(x,y,w,b,lambdaa):
    m,n=x.shape
    loss = 0.0
    dj_dw = np.zeros((n,))
    dj_db = 0.
    for i in range(m):
        f_wb_i = sigmoid(np.dot(x[i],w)+b)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] += (err_i.item() * x[i, j].item())
        dj_db += err_i
    dj_dw /= m
    dj_db /= m
    for j in range(n):
        dj_dw[j] += ((lambdaa/m)*w[j])
    return dj_db,dj_dw

In [16]:
def gradient_descent_reg(x,y,w_in,b_in,alpha,num_iters):
    j_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    for i in range(num_iters):
        dj_db,dj_dw = compute_logistic_gradient_reg(x,y,w,b,0.7)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        j_history.append(compute_logistic_cost_reg(x,y,w,b,0.7))
        if i%math.ceil(num_iters/10)==0:
            print(f"Iterations: {i}, Cost: {j_history[-1]}")
    return w,b,j_history

In [31]:
w_tmp = np.random.rand(x_train.shape[1]).reshape(-1,)-0.5
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp,dj_dw_tmp = compute_logistic_gradient_reg(x_train,y_train,w_tmp,b_tmp,lambda_tmp)
print(f"dj_db: {dj_db_tmp} \ndj_dw: {dj_dw_tmp.tolist()}")
w_out, b_out,j_history = gradient_descent_reg(x_train,y_train,w_tmp,b_tmp,0.001,10000)
print(f"Values of weight and bias: {w_out},{b_out}")

dj_db: [0.07945318] 
dj_dw: [-0.34391301605079727, -0.36487328730809104, -0.3426997465381249, -0.34778788904293145, -0.3504115281067864]
Iterations: 0, Cost: [0.09608153]
Iterations: 1000, Cost: [0.04027916]
Iterations: 2000, Cost: [0.03030988]
Iterations: 3000, Cost: [0.02675738]
Iterations: 4000, Cost: [0.02513996]
Iterations: 5000, Cost: [0.02434578]
Iterations: 6000, Cost: [0.02397102]
Iterations: 7000, Cost: [0.02383441]
Iterations: 8000, Cost: [0.02384284]
Iterations: 9000, Cost: [0.02394381]
Values of weight and bias: [0.73476259 0.43728523 1.09129179 1.06772456 0.57587677],[0.29201217]


In [32]:
def predict(x, w, b):
    z = np.dot(x, w) + b
    prob = sigmoid(z)
    return 1 if prob >= 0.5 else 0, prob

In [34]:
new_student = np.array([3,71,102,93,94])
new_student = z_score_normalization(new_student)
prediction, prob = predict(new_student,w_out,b_out)
print(f"Prediction: {'Pass' if prediction==1 else 'Fail'}")
print(f"Probability of Passing the Exam: {prob}")

Prediction: Pass
Probability of Passing the Exam: [0.66539536]
